In [2]:
# Step 1: Import necessary libraries
import os
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# Define the path to your dataset
dataset_dir = r'D:\SEM_V\Industry_project\visual_studio_code\wheat_merged_dataset'  # Update this to your local dataset path
categories = ['Healthy', 'wheat_leaf_rust', 'wheat_loose_smut']

# Create directories for each split and category
def create_dirs(base_dir, class_names):
    for class_name in class_names:
        os.makedirs(os.path.join(base_dir, class_name), exist_ok=True)

base_dir = r'D:\SEM_V\Industry_project\visual_studio_code'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

create_dirs(train_dir, categories)
create_dirs(val_dir, categories)
create_dirs(test_dir, categories)

# Split the dataset into Train/Validation/Test sets
for category in categories:
    img_dir = os.path.join(dataset_dir, category)
    images = os.listdir(img_dir)
    
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)
    train_images, val_images = train_test_split(train_images, test_size=0.2, random_state=42)
    
    for image in train_images:
        shutil.copy(os.path.join(img_dir, image), os.path.join(train_dir, category, image))
    
    for image in val_images:
        shutil.copy(os.path.join(img_dir, image), os.path.join(val_dir, category, image))
    
    for image in test_images:
        shutil.copy(os.path.join(img_dir, image), os.path.join(test_dir, category, image))

# Data Preprocessing (Rescale only)
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    train_dir, target_size=(150, 150), batch_size=32, class_mode='categorical')

validation_generator = datagen.flow_from_directory(
    val_dir, target_size=(150, 150), batch_size=32, class_mode='categorical')

test_generator = datagen.flow_from_directory(
    test_dir, target_size=(150, 150), batch_size=32, class_mode='categorical')

# Updated CNN Model
model = Sequential()

# First Convolutional Block
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Second Convolutional Block
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Third Convolutional Block
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Fully Connected Layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.6))  # Increased Dropout to prevent overfitting

# Output Layer (3 classes)
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Implement Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the Model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=30,  
    callbacks=[early_stopping]
)

# Evaluate the Model on the Test Set
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy: {test_acc:.4f}')

# Print Final Training Accuracy
train_acc = history.history['accuracy'][-1]
print(f'Final Training Accuracy: {train_acc:.4f}')

# Save the Model
model.save('wheat_leaf_nn_model.h5')


Found 2560 images belonging to 3 classes.
Found 639 images belonging to 3 classes.
Found 796 images belonging to 3 classes.
Epoch 1/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 190s 2s/step - accuracy: 0.6935 - loss: 1.9013 - val_accuracy: 0.4013 - val_loss: 5.2830
Epoch 2/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.3548 - val_loss: 5.7757
Epoch 3/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 153s 2s/step - accuracy: 0.8745 - loss: 0.4208 - val_accuracy: 0.3997 - val_loss: 7.4596
Epoch 4/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.3871 - val_loss: 7.8752
Epoch 5/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 151s 2s/step - accuracy: 0.9090 - loss: 0.2706 - val_accuracy: 0.3947 - val_loss: 6.7164
Epoch 6/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5161 - val_loss: 4.6657
Epoch 7/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 155s 2s/step - accuracy: 0.9459 - loss: 0.1559 - val_

Test Accuracy: 0.9246
Final Training Accuracy: 0.9895


In [1]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load the model
model = load_model('wheat_leaf_nn_model.h5')

# Load and preprocess the image
img_path = "loose_1.jpeg"  # Update with your image path
img = image.load_img(img_path, target_size=(150, 150))  # Ensure correct size
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize as done during training

# Make prediction
predictions = model.predict(img_array)

# Print raw prediction probabilities
print(f"Raw prediction probabilities: {predictions}")

# Get predicted class
predicted_class_idx = np.argmax(predictions, axis=1)

# Output prediction
class_labels = ['Healthy', 'wheat_leaf_rust', 'wheat_loose_smut']  # Ensure this matches the training order
predicted_class = class_labels[predicted_class_idx[0]]

print(f'Predicted Disease: {predicted_class}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 387ms/step
Raw prediction probabilities: [[7.0284150e-04 2.6831345e-03 9.9661404e-01]]
Predicted Disease: wheat_loose_smut
